In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import fetch_california_housing

In [2]:
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

train_adult = pd.read_csv("DataSets/census/adult.data", header=None, names=columns, sep=",", na_values=" ?", skipinitialspace=True)
test_adult = pd.read_csv("DataSets/census/adult.test", header=0, names=columns, sep=",", na_values=" ?", skipinitialspace=True, comment='|')
test_adult['income'] = test_adult['income'].str.replace('.', '', regex=False)

data_adult = pd.concat([train_adult, test_adult], ignore_index=True).dropna()


In [13]:
housing = fetch_california_housing()

X = housing.data
y = housing.target

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print("\nFeature names:")
print(housing.feature_names)

Features (X) shape: (20640, 8)
Target (y) shape: (20640,)

Feature names:
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


# Part 1: Feature-Level Interpretability (30 marks)  
You will use the California Housing and the Adult Census Income datasets in this part. You 
should train one feed-forward neural network for each dataset and apply the following 
interpretability techniques:

In [7]:
# Adult Census Income Dataset pre-processing and neural network model

X_adult = data_adult.drop("income", axis=1)
y_adult = (data_adult["income"] == ">50K").astype(int)

# Identify categorical and numerical columns
cat_cols = X_adult.select_dtypes(include=['object']).columns
num_cols = X_adult.select_dtypes(exclude=['object']).columns

# Encode categorical & scale numeric
ct = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('scale', StandardScaler(), num_cols)
])

X_processed_adult = ct.fit_transform(X_adult)
X_train_adult, X_val_adult, y_train_adult, y_val_adult = train_test_split(X_processed_adult, y_adult, test_size=0.2, random_state=42)

input_dim = X_train_adult.shape[1]

# The feed-forward neural network model
model = Sequential([
    Dense(128, activation='relu', input_dim=input_dim),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(
    X_train_adult, y_train_adult,
    validation_data=(X_val_adult, y_val_adult),
    epochs=10,
    batch_size=256,
    verbose=1
)

# Evaulate the model
loss, acc = model.evaluate(X_val_adult, y_val_adult, verbose=0)
print(f"Validation Accuracy: {acc:.4f}")

Epoch 1/10


c:\Users\jamie\OneDrive\Desktop\Interpretable AI Project\.venv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


153/153 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7881 - loss: 0.4447 - val_accuracy: 0.8539 - val_loss: 0.3168
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8530 - loss: 0.3196 - val_accuracy: 0.8552 - val_loss: 0.3121
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8553 - loss: 0.3113 - val_accuracy: 0.8576 - val_loss: 0.3095
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8589 - loss: 0.3074 - val_accuracy: 0.8575 - val_loss: 0.3077
Epoch 5/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8579 - loss: 0.3073 - val_accuracy: 0.8584 - val_loss: 0.3081
Epoch 6/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8606 - loss: 0.3047 - val_accuracy: 0.8594 - val_loss: 0.3061
Epoch 7/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8591 - loss: 0.3059 - val_accuracy: 0.8584 - val_loss: 0.3073
Epoch 8/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8607 - loss: 0.3010 - val_accuracy: 0.8598 - val_

"\n\nfrom scikeras.wrappers import KerasClassifier\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.preprocessing import OneHotEncoder, StandardScaler\nfrom sklearn.inspection import PartialDependenceDisplay\nimport matplotlib.pyplot as plt\n\n# Wrap the Keras model\ndef create_model():\n    model = Sequential([\n        Dense(128, activation='relu', input_dim=input_dim),\n        Dropout(0.3),\n        Dense(64, activation='relu'),\n        Dropout(0.3),\n        Dense(1, activation='sigmoid')\n    ])\n    model.compile(optimizer=Adam(learning_rate=0.001),\n                  loss='binary_crossentropy', metrics=['accuracy'])\n    return model\n\n# Wrap in scikit-learn compatible estimator\nsklearn_model = KerasClassifier(model=create_model, epochs=10, batch_size=256, verbose=0)\nsklearn_model.fit(X_train_adult, y_train_adult)  # Train the model\n"

In [14]:
# California Housing Dataset pre-processing and neural network model

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

print(f"Total data points: {len(X)}")
print(f"Training data points: {len(X_train)}")
print(f"Validation data points: {len(X_valid)}")
print(f"Test data points: {len(X_test)}")


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

print("Data successfully scaled")
print(f"Original mean (first feature): {X_train[:, 0].mean():.4f}")
print(f"Scaled mean (first feature): {X_train_scaled[:, 0].mean():.4f}")

n_features = X_train_scaled.shape[1]

model = keras.Sequential([

    layers.Dense(64, activation="relu", input_shape=[n_features]),
    layers.Dropout(0.2),
    
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.2),

    layers.Dense(16, activation="relu"),
    
    layers.Dense(1)
])

model.summary()

model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["mean_absolute_error"]
)

print("Model compiled with MSE as the loss function and Dropout Layers and a custom Adam.")

print("--- Starting Model Training ---")

early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_valid_scaled, y_valid),
    callbacks=[early_stopping],
    verbose=1
)

print("--- Model Training Finished ---")

print("--- Evaluating Model on Test Set ---")

results = model.evaluate(X_test_scaled, y_test, verbose=0)

final_mse = results[0]
final_mae = results[1]

print(f"Final Test Set MSE (Mean Squared Error): {final_mse:.4f}")
print(f"Final Test Set MAE (Mean Absolute Error): {final_mae:.4f}")

Total data points: 20640
Training data points: 13209
Validation data points: 3303
Test data points: 4128
Data successfully scaled
Original mean (first feature): 3.8689
Scaled mean (first feature): -0.0000


c:\UoM\Year 3\ARI3205 Interpretable AI for DL Models\interpretableaiproject\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

Model compiled with MSE as the loss function and Dropout Layers and a custom Adam.
--- Starting Model Training ---
Epoch 1/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.0097 - mean_absolute_error: 0.7006 - val_loss: 0.4684 - val_mean_absolute_error: 0.4839
Epoch 2/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5490 - mean_absolute_error: 0.5315 - val_loss: 0.4474 - val_mean_absolute_error: 0.4681
Epoch 3/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4735 - mean_absolute_error: 0.4941 - val_loss: 0.4393 - val_mean_absolute_error: 0.4642
Epoch 4/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4564 - mean_absolute_error: 0.4805 - val_loss: 0.4288 - val_mean_absolute_error: 0.4556
Epoch 5/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4306 - mean_absolute_error: 0.4712 - val_loss: 0.4103 - val_mean_absolute_error: 0.4483
Epoch 6/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4096 - mean_absolute_error: 0.4584 - val_loss: 0.3859 - val_mean_ab

## 1. Partial Dependence Plots (PDP) and Individual Conditional Expectation (ICE) plots (7 marks) 
### a. Use PDP to examine the average effect of at least two features. 

In [11]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# Select only numeric features for PDP
X_train_numeric = X_train_adult[:, -len(num_cols):]  # last columns are scaled numeric
numeric_feature_names = list(num_cols)  # ['age', 'fnlwgt', 'education-num', ...]

# Wrap your trained model
class KerasWrapper:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def predict_proba(self, X):
        probs = self.model.predict(X, verbose=0).flatten()
        return np.vstack([1 - probs, probs]).T

wrapped_model = KerasWrapper(model)
wrapped_model.fit(X_train_numeric)

# Pick features: 'age' and 'hours-per-week'
features_for_pdp = ['age', 'hours-per-week']
feature_indices = [numeric_feature_names.index(f) for f in features_for_pdp]

# Plot PDP
PartialDependenceDisplay.from_estimator(
    wrapped_model,
    X_train_numeric,
    features=feature_indices,
    feature_names=numeric_feature_names,
    grid_resolution=20
)
plt.suptitle("Partial Dependence: Education × Hours-per-week", fontsize=14)
plt.tight_layout()
plt.show()


NotFittedError: This KerasWrapper instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
#California Housing Data Set

### b. Use ICE plots to explore individual predictions for at least two features. 

### c. Explain what insights PDP and ICE give about the model’s behaviour.

## 2. Permutation Feature Importance (PFI) (7 marks) 
### a. Use PFI to identify the most important features in the model. 


### b. Explain what the term “important” means when using the PFI method. 

## 3. Accumulated Local Effects (ALE) (9 marks) 
### a. Implement ALE plots to investigate the local effects of feature changes. 

### b. Compare ALE with PDP and discuss any differences in the interpretability of these techniques.

## 4. Global Surrogates (7 marks) 
### a. Build an interpretable model to approximate the predictions of the feed-forward neural network model. 

### b. Analyse the surrogate model's effectiveness and discuss when such approximations are helpful.